In [69]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/bitcoin-tweet-dataset-for-sentiment-analyses/Bitcoin tweet dataset for SA.csv


In [70]:
import nltk 
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.svm import NuSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn import metrics

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

!pip install tweepy
import tweepy as tw
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json

import itertools

!pip install coinbase
from coinbase.wallet.client import Client

!pip install newsapi-python
from newsapi import NewsApiClient

!pip install yfinance 
import yfinance as yf
import datetime
from datetime import date

[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [71]:
df = pd.read_csv('/kaggle/input/bitcoin-tweet-dataset-for-sentiment-analyses/Bitcoin tweet dataset for SA.csv')
df.drop('Unnamed: 0', axis=1, inplace=True) # this was done because a default column titled "Unnamed: 0" was appearing as the first column 
df.dropna(inplace=True) # dropping NaN values
df.head()

,Date,Tweet,Screen_name,Source,Link,Sentiment,sent_score,New_Sentiment_Score,New_Sentiment_State
0,Fri Mar 23 00:40:32 +0000 2018,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",myresumerocket,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['neutral'],0.0,0.000000,0.0
1,Fri Mar 23 00:40:34 +0000 2018,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,BitMocro,[u'Bitcoin'],"<a href=""http://twitter.com/download/android"" ...",['neutral'],0.0,0.000000,0.0
2,Fri Mar 23 00:40:35 +0000 2018,RT @tippereconomy: Another use case for #block...,hojachotopur,"[u'blockchain', u'Tipper', u'TipperEconomy']","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive'],1.0,0.136364,1.0
3,Fri Mar 23 00:40:36 +0000 2018,free coins https://t.co/DiuoePJdap,denies_distro,[],"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",['positive'],1.0,0.400000,1.0
4,Fri Mar 23 00:40:36 +0000 2018,RT @payvxofficial: WE are happy to announce th...,aditzgraha,[],"<a href=""http://twitter.com/download/android"" ...",['positive'],1.0,0.468182,1.0


# Data Wrangling

In [72]:
df.columns

Index(['Date', 'Tweet', 'Screen_name', 'Source', 'Link', 'Sentiment',
       'sent_score', 'New_Sentiment_Score', 'New_Sentiment_State'],
      dtype='object')

In [73]:
# only keeping "Tweet" and "New_Sentiment_State" column
df.drop(['Screen_name', 'Source', 'Link', 'Date', 'Sentiment', 'sent_score', 'New_Sentiment_Score'], axis=1, inplace=True)
df.head(10)

,Tweet,New_Sentiment_State
0,"RT @ALXTOKEN: Paul Krugman, Nobel Luddite. I h...",0.0
1,@lopp @_Kevin_Pham @psycho_sage @naval But @Pr...,0.0
2,RT @tippereconomy: Another use case for #block...,1.0
3,free coins https://t.co/DiuoePJdap,1.0
4,RT @payvxofficial: WE are happy to announce th...,1.0
5,Copy successful traders automatically with Bit...,1.0
6,RT @bethereumteam: We're revealing our surpris...,1.0
7,One click to start mining cryptocurrencies tog...,0.0
8,"first speaker @digitsu\n\n""how we can get bitc...",1.0
9,@p0nd3ea Bitcoin wasn't built to live on excha...,1.0


In [74]:
df.groupby(['New_Sentiment_State']).count() # they are not equal in number so our model is biased again we will need to make them all equal i.e 6565 each

,Tweet
New_Sentiment_State,
-1.0,6565
0.0,21742
1.0,22545


In [75]:
pos = df.loc[df['New_Sentiment_State'] > 0]
neg = df.loc[df['New_Sentiment_State'] < 0]
neutral = df.loc[df['New_Sentiment_State'] == 0]
pos = pos.sample(n=6565)
neutral = neutral.sample(n=6565)

frames = [pos,neutral,neg]
res1 = pd.concat(frames)

df = res1.sample(frac=1)
df.head()

,Tweet,New_Sentiment_State
41081,How to Kill Bitcoin? https://t.co/j3B0CrVUk4 h...,0.0
28501,#MoneyHack Why is bitcoin better than money ht...,1.0
10595,RT @SubstratumNet: For all those who wanted to...,0.0
16435,"Cheap mining, sign up to Genesis Mining https:...",1.0
36481,Ramencoin airdrop is live now. https://t.co/9Y...,1.0


In [76]:
df.groupby(['New_Sentiment_State']).count()

,Tweet
New_Sentiment_State,
-1.0,6565
0.0,6565
1.0,6565


# Preprocessing

In [77]:
stemmer = SnowballStemmer("english")

def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

# Tokenize and lemmatize
def preprocess(text):
    
    result = []
    
    for token in gensim.utils.simple_preprocess(text) :
        
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            
            # TODO: Apply lemmatize_stemming() on the token, then add to the results list
            result.append(lemmatize_stemming(token))
    
    return result

def further_processing(processed_docs):
    # this is the step where each word is given a number from which it can be recognized
    tokenizer = Tokenizer(oov_token='<00V>')
    tokenizer.fit_on_texts(processed_docs)
    word_index = tokenizer.word_index
    #print(word_index)

    # here the tokenized numbers are put into sequences as compared to the post titles
    sequences = tokenizer.texts_to_sequences(processed_docs)
    

    # here we make all the sentences of equal length
    padded = pad_sequences(sequences, padding='post', maxlen=18)
    return padded

processed_docs = df['Tweet'].map(preprocess)
padded = further_processing(processed_docs)

# Training and Testing the models

In [78]:
X = padded
y = df['New_Sentiment_State']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state = 42)

# this function helps in plotting the accuracy of the models
name_arr = []
acc_arr = []
def lst_append(name, acc):
  name_arr.append(name)
  acc_arr.append(acc)
  
def model_perf(model, name):
  model.fit(X_train, y_train)
  preds = model.predict(X_test)
  acc = round(accuracy_score(y_test, preds), 5)*100
  acc_lst = lst_append(name, acc)
  print('======================================')
  print('Accuracy ' + f'{name}' + ':', acc,'\n')
  print(classification_report(y_test, preds))
  print('======================================')

RFC = RandomForestClassifier(criterion = 'gini')
model_perf(RFC, 'Random Forest')

SGDC = SGDClassifier(max_iter=1000, tol=1e-3)
model_perf(SGDC, 'Stochastic Gradient Descent')

svm = SVC()
model_perf(svm, 'Support Vector Machine')

LSVC = LinearSVC(random_state=0, tol=1e-5)
model_perf(LSVC, 'Linear Support Vector Classification')

GNB = GaussianNB()
model_perf(GNB, 'Naive Bayes')

BNB = BernoulliNB()
model_perf(BNB, 'Bernoulli Naive Bayes')

MNB = MultinomialNB()
model_perf(MNB, 'Multinomial Naive Bayes')

AdaB = AdaBoostClassifier(n_estimators=100, random_state=0)
model_perf(AdaB, 'AdaBoost')

LGBM = LGBMClassifier(random_state=5)
model_perf(LGBM, 'Light Gradient Boosting Machine')

GBC = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
model_perf(GBC, 'Gradient Boost Classifier')

Accuracy Random Forest: 81.977 

              precision    recall  f1-score   support

        -1.0       0.90      0.81      0.85      1924
         0.0       0.76      0.86      0.81      2010
         1.0       0.82      0.78      0.80      1975

    accuracy                           0.82      5909
   macro avg       0.83      0.82      0.82      5909
weighted avg       0.83      0.82      0.82      5909

Accuracy Stochastic Gradient Descent: 34.862 

              precision    recall  f1-score   support

        -1.0       0.46      0.22      0.30      1924
         0.0       0.33      0.76      0.46      2010
         1.0       0.26      0.06      0.09      1975

    accuracy                           0.35      5909
   macro avg       0.35      0.35      0.28      5909
weighted avg       0.35      0.35      0.29      5909

Accuracy Support Vector Machine: 46.387 

              precision    recall  f1-score   support

        -1.0       0.45      0.62      0.52      1924
       

/opt/conda/lib/python3.7/site-packages/sklearn/svm/_base.py:977: ConvergenceWarning:

Liblinear failed to converge, increase the number of iterations.



Accuracy Linear Support Vector Classification: 38.67 

              precision    recall  f1-score   support

        -1.0       0.45      0.38      0.41      1924
         0.0       0.30      0.05      0.09      2010
         1.0       0.37      0.73      0.49      1975

    accuracy                           0.39      5909
   macro avg       0.37      0.39      0.33      5909
weighted avg       0.37      0.39      0.33      5909

Accuracy Naive Bayes: 34.27 

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00      1924
         0.0       0.34      0.99      0.51      2010
         1.0       0.54      0.01      0.03      1975

    accuracy                           0.34      5909
   macro avg       0.29      0.34      0.18      5909
weighted avg       0.30      0.34      0.18      5909

Accuracy Bernoulli Naive Bayes: 39.736 

              precision    recall  f1-score   support

        -1.0       0.38      0.55      0.45      1924
   

In [79]:
dict2 = {'Name of the model': name_arr, 'Accuracy': acc_arr}
perf = pd.DataFrame(dict2)
fig = px.bar(perf, x='Name of the model', y='Accuracy')
fig.show()

In [80]:
VOCAB_SIZE = 20762
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 64),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, None, 64)          1328768   
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 33        
Total params: 1,341,217
Trainable params: 1,341,217
Non-trainable params: 0
_________________________________________________________________


In [81]:
train_data = padded[0:4595]
train_labels = df['New_Sentiment_State'][0:4595]
test_data = padded[4595:6565]
test_labels = df['New_Sentiment_State'][4595:6565]

model.compile(loss="binary_crossentropy",optimizer="rmsprop",metrics=['acc'])

history = model.fit(train_data, train_labels, epochs=10, validation_split=0.2)

Epoch 1/10
115/115 [==============================] - 4s 19ms/step - loss: -0.3308 - acc: 0.3831 - val_loss: -2.0812 - val_acc: 0.5321
Epoch 2/10
115/115 [==============================] - 2s 14ms/step - loss: -2.8975 - acc: 0.5827 - val_loss: -3.2335 - val_acc: 0.5495
Epoch 3/10
115/115 [==============================] - 2s 13ms/step - loss: -4.4015 - acc: 0.6156 - val_loss: -4.2132 - val_acc: 0.5637
Epoch 4/10
115/115 [==============================] - 2s 14ms/step - loss: -5.6694 - acc: 0.6431 - val_loss: -5.0259 - val_acc: 0.5517
Epoch 5/10
115/115 [==============================] - 2s 14ms/step - loss: -7.2176 - acc: 0.6283 - val_loss: -5.9976 - val_acc: 0.5528
Epoch 6/10
115/115 [==============================] - 2s 14ms/step - loss: -8.1409 - acc: 0.6541 - val_loss: -6.9332 - val_acc: 0.5473
Epoch 7/10
115/115 [==============================] - 2s 13ms/step - loss: -9.2405 - acc: 0.6625 - val_loss: -7.8341 - val_acc: 0.5452
Epoch 8/10
115/115 [==============================] - 2

In [82]:
results = model.evaluate(test_data, test_labels)
print(results)

62/62 [==============================] - 0s 3ms/step - loss: -11.5584 - acc: 0.5437
[-11.558425903320312, 0.5436547994613647]


# Feeding real-time data to our models

In [83]:
# Before we take data from Twitter we need to know the top 10 cryptocurrencies based on market capitalization for which we use the Coinbase API
url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
parameters = {
  'start':'1',
  'limit':'5000',
  'convert':'USD'
}
headers = {
  'Accepts': 'application/json',
  'X-CMC_PRO_API_KEY': '975df79a-7413-495e-8f06-67564fb215eb',
}

session = Session()
session.headers.update(headers)

try:
  response = session.get(url, params=parameters)
  data = json.loads(response.text)
  #print(data)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

technicals = []
max_supply = []
circulating_supply = []
symbol = []
search_words = []
for i in range(0,10):
    technicals.append(data['data'][i]['quote']['USD'])
    max_supply.append(data['data'][i]['max_supply'])
    circulating_supply.append(data['data'][0]['circulating_supply'])
    symbol.append(data['data'][i]['symbol'])
    search_words.append(data['data'][i]['name'])
    
df_crypto = pd.DataFrame()
df_crypto['Cryptocurrency'] = search_words
df_crypto['Symbol'] = symbol

price = [] 
volume_24h = []
percent_change_1h = []
percent_change_24h = []
percent_change_7d = []
percent_change_30d = []
percent_change_60d = []
percent_change_90d = []
market_cap = []
market_cap_dominance = []
fully_diluted_market_cap = []
last_updated = []

for i in technicals:
    count = 0
    for j in i:
        k = i.get(j)
        count = count + 1
        if count == 1:
            price.append(k)
        if count == 2:
            volume_24h.append(k)
        if count == 3:
            percent_change_1h.append(k)
        if count == 4:
            percent_change_24h.append(k)
        if count == 5:
            percent_change_7d.append(k)
        if count == 6:
            percent_change_30d.append(k)
        if count == 7:
            percent_change_60d.append(k)
        if count == 8:
            percent_change_90d.append(k)
        if count == 9:
            market_cap.append(k)
        if count == 10:
            market_cap_dominance.append(k)
        if count == 11:
            fully_diluted_market_cap.append(k)
        if count == 12:
            last_updated.append(k)

df_crypto['Price'] = price
df_crypto['Volume in 24h'] = volume_24h
df_crypto['% change in 1h'] = percent_change_1h
df_crypto['% change in 24h'] = percent_change_24h
df_crypto['% change in 7d'] = percent_change_7d
df_crypto['% change in 30d'] = percent_change_30d
df_crypto['% change in 60d'] = percent_change_60d
df_crypto['% change in 90d'] = percent_change_90d
df_crypto['Market Cap'] = market_cap
df_crypto['Fully diluted Market Cap'] = fully_diluted_market_cap
df_crypto['Last updated'] = last_updated
df_crypto['Circulating Supply'] = circulating_supply
df_crypto['Max Supply'] = max_supply

In [84]:
# getting tweets from Twitter using it's API
consumer_key= 'uLPC3KfMtGFcEeq4CxEOohZeg'
consumer_secret= 'tywsJRvcr2zz5ICg7bkadbSIIjhGFmAlOLjJECjPqMfaRuwc1T'
access_token= '1300465599823314944-VkC6tWnEUrbxTZ1wYpWIxbc8LQCPNL'
access_token_secret= 'DDiF0cmidxoQlT2rgEUCGkP4E2DI8PBwz6WMS5QL51zOG'
auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)

date_since = '2021-04-01'
tweet_text = []
date_time = []
location = []

for words in search_words:
  tweets = tw.Cursor(api.search,
              q=search_words,
              lang="en",
              since=date_since).items(10000)
  for tweet in tweets:
    str1 = tweet.text
    str2 = tweet.created_at
    str3 = tweet.user.location
    tweet_text.append(str1)
    date_time.append(str2)
    location.append(str3)

df_twitter = pd.DataFrame()
df_twitter['Tweets'] = tweet_text
df_twitter['Created at'] = date_time
df_twitter['Location'] = location

df_twitter.head()

,Tweets,Created at,Location
0,RT @SpeedyCalls: Top 10 Cryptoassets by Market...,2021-08-30 18:51:16,Albuquerque
1,RT @SpeedyCalls: Top 10 Cryptoassets by Market...,2021-08-30 18:48:49,
2,Top 10 Cryptoassets by Market Cap:\n1 #Bitcoi...,2021-08-30 18:31:27,Global
3,Top 10 Cryptoassets by Market Cap:\n1 #Bitcoi...,2021-08-29 18:29:15,Global
4,RT @SpeedyCalls: Top 10 Cryptoassets by Market...,2021-08-30 18:51:16,Albuquerque


In [85]:
# preprocessing the data
processed_docs = df_twitter['Tweets'].map(preprocess)
padded_1 = further_processing(processed_docs)

In [86]:
def just_for_prediction(model, tweets):
    prediction = model.predict(tweets)
    return prediction

# we will only be using these 4 classifiers as the others have a very low accuracy 
lst1 = just_for_prediction(RFC, padded_1)
lst2 = just_for_prediction(LGBM, padded_1)
lst3 = just_for_prediction(AdaB, padded_1)
lst4 = just_for_prediction(GBC, padded_1)
lst5 = range(len(lst1))

df1 = pd.DataFrame({'Tweet ID': lst5, 'Random Forest': lst1, 'Light Gradient Boosting Machine':lst2, 'AdaBoost': lst3, 'Gradient Boost Classifier':lst4})
df1.set_index('Tweet ID', inplace=True)
df1.head()

,Random Forest,Light Gradient Boosting Machine,AdaBoost,Gradient Boost Classifier
Tweet ID,,,,
0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0
2,0.0,0.0,-1.0,-1.0
3,0.0,0.0,-1.0,-1.0
4,0.0,0.0,0.0,0.0


In [87]:
lst0 = []
negative = 0
neutral = 0 
positive = 0
for (a,b,c,d) in zip(lst1,lst2,lst3,lst4):
    lst6 = list([a,b,c,d])
    # print(lst6)
    if lst6.count(-1)>lst6.count(1) and lst6.count(-1)>lst6.count(0):
        str1 = 'Negative'
        negative = negative + 1 
    elif lst6.count(0)>lst6.count(1) and  lst6.count(0)>lst6.count(-1):
        str1 = 'Neutral'
        neutral = neutral + 1
    else:
        str1 = 'Positive'
        positive = positive + 1
    lst0.append(str1)

df2 = pd.DataFrame({'Tweet ID': lst5, 'Prediction': lst0})
lst7 = ['Negative', 'Neutral', 'Positive']
lst8 = [negative, neutral, positive]
df3 = pd.DataFrame({'Sentiment': lst7, 'Count of Sentiment':lst8})
fig = px.bar(df3, x='Sentiment', y='Count of Sentiment')
fig.show()

In [88]:
# news articles regarding the cryptocurrencies
# Init
newsapi = NewsApiClient(api_key='56885df3e9f04b6a9762a4b1a33f9f1e')

today = date.today()
d = datetime.timedelta(days = 5)
a = today - d

# /v2/everything
all_articles = newsapi.get_everything(q='Bitcoin',
                                      sources='axios, bloomberg, business-insider, crypto-coins-news, engadget, financial-post, google-news, hacker-news, mashable, next-big-futre, recode, reuters, techcrunch-cn, techradar, wired, the-wall-street-journal, bbc-news, fortune, abc-news, al-jazeera-english,ars-technica, australian-financial-review, breitbart-news, associated-press, buzzfeed, cbs-news, fox-news, independent, medical-news-today, national-review, nbc-news, new-scientist, news24, news-com-au, newsweek, new-york-magazine, politico, polygon, reddit-r-all, techcrunch, the-american-conservative, the-hindu, the-huffington-post, the-lad-bible, the-next-web, the-times-of-india, the-washington-post, the-washington-times, time, vice-news',
                                      domains='bbc.co.uk,techcrunch.com',
                                      from_param='2021-18-20',
                                      to='2021-08-30',
                                      language='en',
                                      sort_by='relevancy',
                                      page=1)

# /v2/top-headlines/sources
sources = newsapi.get_sources()

In [89]:
# function to plot their OHLC graphs
def func(name):
    today = date.today()
    tickerSymbol = name+'-USD'
    tickerData = yf.Ticker(tickerSymbol)
    df = tickerData.history(period='1d', start='2000-01-01', end=today)
    df.reset_index(inplace=True)
     
    fig = go.Figure(data=go.Ohlc(x=df['Date'],
                    open=df['Open'],
                    high=df['High'],
                    low=df['Low'],
                    close=df['Close']))
    fig.show()